**Problem 3: Credit Card Fraud | SVM**

In [89]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
file_path = '/content/drive/MyDrive/creditcard.rar'


In [91]:
import patoolib

# Specify the path to the downloaded RAR file
rar_path = file_path

# Extract all files from the RAR archive
patoolib.extract_archive(rar_path, outdir='/content/')

patool: Extracting /content/drive/MyDrive/creditcard.rar ...
patool: running /usr/bin/unrar x -- /content/drive/MyDrive/creditcard.rar
patool:     with cwd='/content/'


PatoolError: ignored

a) Load data set and split the dataset into 80% training data and 20% validation data then Implement the SVM algorithm on the training data and tune the parameters to achieve the best performance

In [101]:
import pandas as pd


df = pd.read_csv('creditcard.csv')


class_0 = df[df['Class'] == 0]
class_1 = df[df['Class'] == 1]

num_samples_class_0 = int(0.99 * 10000)  # 99% of 10,000
num_samples_class_1 = 10000 - num_samples_class_0  # 1% of 10,000

sample_0 = class_0.sample(n=num_samples_class_0, random_state=42)


sample_1 = class_1.sample(n=num_samples_class_1, random_state=42)

# Combine
sample_df = pd.concat([sample_0, sample_1])

# Shuffle the combined sample
credit_sample = sample_df.sample(frac=1, random_state=42)

print(credit_sample)

            Time        V1        V2        V3        V4        V5        V6  \
27322    34501.0 -1.413437  0.581796  1.644620 -1.496490 -0.235255 -0.056796   
258660  158756.0 -0.174574  1.024532 -1.436532 -0.502308 -0.131019  0.388579   
116255   74203.0  1.271681  0.259960 -0.080704  0.449564  0.195874 -0.129810   
148167   89505.0 -0.272799  1.055515 -1.041490 -0.737320  1.567775 -1.023466   
93534    64472.0  1.122450 -0.461956  1.248189  0.116328 -1.324130 -0.264058   
...          ...       ...       ...       ...       ...       ...       ...   
274767  166195.0 -0.074949 -0.581335  0.003563 -3.129617  0.036601 -0.499372   
174211  121820.0  0.315677  0.792154 -1.980506  0.166049  2.757871  3.954494   
108876   71116.0 -0.778253  0.130171  2.330368  1.743399 -0.602476  0.995023   
88007    61944.0 -2.242599  0.091694  1.335811  0.234568  1.097002  0.989207   
279854  169135.0 -1.196644  1.930087 -3.515438 -1.333896  3.579285  2.453577   

              V7        V8        V9  .

In [93]:
print(len(credit_sample))

10000


b) What is one-class SVM and how can it be used in outlier detection? run the one-class SVM on training data. (Set the nu, kernel and gamma).

In [94]:
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X = credit_sample.drop('Class', axis=1)
target = credit_sample['Class']

# Convert labels to -1 (outliers) and 1 (normal)
target[target == 0] = -1

# Feature Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and validation sets
X_train, X_val, target_train, target_val = train_test_split(X, target, test_size=0.2, random_state=42)



# One-Class SVM model
one_class_svm = OneClassSVM()

#parametr tuning
param_grid = {
    'gamma': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'nu': [0.01, 0.05, 0.1, 0.2]
}


<ipython-input-94-c0962b906d64>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target[target == 0] = -1


In [95]:

# Perform grid search to find the best parameters
grid_search = GridSearchCV(one_class_svm, param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, target_train)

# Get the best parameters and model
best_params = grid_search.best_params_


print("best parametrs : \n")
print(best_params)

best parametrs : 

{'gamma': 10, 'kernel': 'rbf', 'nu': 0.2}


c) Make outlier predictions on the validation dataset. Which of the metrics such as precision, recall, etc. can show how accurately our model was able to detect outlier data? Explain your reasons.

In [98]:
import pandas as pd
from sklearn.svm import OneClassSVM



# Instantiate the One-Class SVM model
one_class_svm = OneClassSVM(nu=0.2, kernel='rbf', gamma=10)

# Fit the One-Class SVM on the training data
one_class_svm.fit(X)





OneClassSVM(gamma=10, nu=0.2)

In [100]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
# Predict the labels for the validation data
target_valid_pred = one_class_svm.predict(X_val)





# Compute precision, recall, and F1-score
precision = precision_score(target_val, target_valid_pred, average='weighted')
recall = recall_score(target_val, target_valid_pred, average='weighted')
f1 = f1_score(target_val, target_valid_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.9806657916981805
Recall: 0.4125
F1-score: 0.5735046281519312
